In [1]:
import numpy as np
import random
from PIL import Image
from collections import deque
from keras.models import Sequential
from keras import initializers
from keras.layers import Dense, Conv2D, Dropout, Flatten
from keras.optimizers import Adam, Nadam, Adamax
sizes = (80,80,1)

class Agent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=50000)
        self.gamma = 0.999   # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01  # exploration will not decay futher
        self.epsilon_decay = 0.000995
        self.learning_rate = 0.0001
        self.loss = 0
        self.model = self._build_model()
        self.weight_backup = 'model_weights.h5'

    def _build_model(self):
        model = Sequential()
        model.add(Conv2D(32, kernel_size=8, subsample=(4, 4), activation='relu', kernel_initializer='random_uniform', bias_initializer='zeros', padding='same', input_shape= sizes))#80*80*4
        model.add(Conv2D(64, kernel_size=4, subsample=(2, 2), activation='relu', padding='same'))
        model.add(Conv2D(64, kernel_size=3, subsample=(1, 1), activation='relu', padding='same'))
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dense(self.action_size))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    
    def save_model(self):
            self.model.save(self.weight_backup)

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def remember(self, state, action, reward, new_state, done):
        if len(self.memory) >= 50000:
            self.memory.popleft()
            self.memory.append([state, action, reward, new_state, done])
        else:
            self.memory.append([state, action, reward, new_state, done])    

    def memory_replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        Sample = random.sample(self.memory, batch_size)
        
        winsample = [s for s in self.memory if s[2]== 1.0]
        #print(winsample)
        tuple(winsample)
        if len(winsample) > 1:
            Samplewin = random.sample(winsample, 1)
            Sample += Samplewin
        else:  
            Sample += winsample
        
        inputs = np.zeros((len(Sample), state.shape[1], state.shape[2], state.shape[3])) # minibatch input
        targets = np.zeros((inputs.shape[0], self.action_size))
        
        for i in range(0, len(Sample)):
            sample_state = Sample[i][0]
            sample_action = Sample[i][1]
            sample_reward = Sample[i][2]
            sample_new_state = Sample[i][3]
            sample_done = Sample[i][4]
            
            inputs[i:i+1] = sample_state # slice of inputs setting = to state
            
            targets[i] = self.model.predict(sample_state)
            future_reward = self.model.predict(sample_new_state)
            
            if sample_done:
                targets[i, sample_action] = sample_reward
            elif sample_reward == 1.0:
                targets[i, sample_action] = 10.0
            elif sample_reward == -1.0:
                targets[i, sample_action] = sample_reward
            else:
                targets[i, sample_action] = sample_reward + self.gamma * np.max(future_reward)
            print(sample_action, sample_reward, targets[i, sample_action])
        self.loss += self.model.train_on_batch(inputs, targets)
        print(self.loss)
        if self.epsilon > self.epsilon_min:
            self.epsilon -= self.epsilon_decay
    '''        
    def RGBprocess(new_state):
        processed_observation = Image.fromarray(new_state, 'RGB')
        processed_observation = processed_observation.convert('L')
        processed_observation = processed_observation.resize((80, 80))
        processed_observation = np.array(processed_observation)
        processed_observation = processed_observation.reshape(1, processed_observation.shape[0], processed_observation.shape[1], 1) #1x80x80x1
        return processed_observation
        #stack4.append(processed_observation)
        #if len(stack4) == 4:
            #stack_of_observation = np.stack((processed_observation, processed_observation, processed_observation, processed_observation), axis=2)
            #stack_of_observation = stack_of_observation.reshape(stack_of_observation.shape[0], stack_of_observation.shape[1], stack_of_observation.shape[3], stack_of_observation.shape[2])
            #print(stack_of_observation.shape)
    '''

Using TensorFlow backend.


In [ ]:
import sys
import gym
from gym import wrappers
from scipy import misc
#import cv2

'''
def RGBprocess(raw_img):
        processed_observation = Image.fromarray(raw_img, 'RGB')
        processed_observation = processed_observation.convert('L')
        processed_observation = processed_observation.resize((80, 80))
        processed_observation = np.array(processed_observation)
        processed_observation = processed_observation.reshape(1, processed_observation.shape[0], processed_observation.shape[1], 1) #1x80x80x1
        return processed_observation
'''

def RGBprocess(raw_img):
    grayscale_observation = raw_img.mean(2)
    resized_observation = misc.imresize(grayscale_observation, (80, 80)).astype(np.float32)
    processed_observation = resized_observation.reshape(1, resized_observation.shape[0], resized_observation.shape[1], 1)
    return processed_observation
'''

def RGBprocess(raw_img):
    frame = cv2.cvtColor(cv2.resize(frame, (84, 84)), cv2.COLOR_BGR2GRAY)
    ret, frame = cv2.threshold(frame, 1, 255, cv2.THRESH_BINARY)
    processed_observation = np.reshape(frame, (1, 84, 84, 1))
    return processed_observation
'''

batch_size = 32
#episodes = sys.argv[1] if len(sys.argv) > 1 else 5000
#env_name = sys.argv[2] if len(sys.argv) > 2 else "Pong-v0"

episodes = 1300
env_name = "Pong-v0"
D = 80*80
prev_x = None

env = gym.make(env_name)

env = wrappers.Monitor(env, env_name, force=True)

agent = Agent(env.observation_space.shape, env.action_space.n)

for i_episodes in range(episodes):
    state = env.reset()
    totalreward = 0
    num_actions = 0
    short_mem = []
    done = False
    state = RGBprocess(state)
    while not done:
        if i_episodes % 10 == 0:
            env.render()
        if num_actions % 4 == 0:
            action = agent.act(state)
            new_state, reward, done, info = env.step(action)
            new_state = RGBprocess(new_state)
            new_state_dif = new_state - prev_x if prev_x is not None else new_state
            prev_x = new_state
            agent.remember(state, action, reward, new_state_dif, done)
            totalreward += reward
        num_actions += 1
    agent.memory_replay(batch_size)
    state = new_state_dif
    if done:
        print("{} episode, score = {} ".format(i_episodes + 1, totalreward))
        agent.save_model()

env.close()
gym.upload(env_name, api_key='sk_WRCITkqmTJKYB9hvBk5tPA')

[2017-08-23 11:58:46,903] Making new env: Pong-v0
[2017-08-23 11:58:47,089] Clearing 4 monitor files from previous run (because force=True was provided)
/home/z0m6ie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, kernel_size=8, activation="relu", kernel_initializer="random_uniform", bias_initializer="zeros", padding="same", input_shape=(80, 80, 1..., strides=(4, 4))`
/home/z0m6ie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, kernel_size=4, activation="relu", padding="same", strides=(2, 2))`
/home/z0m6ie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, kernel_size=3, activation="relu", padding="same", strides=(1, 1))`
[2017-08-23 11:58:47,178] Starting new video recorder writing to /home/z0m6ie/Documents/GitHub/Machine_Learning_

1 0.0 0.0
2 0.0 0.341322530866
5 0.0 0.251741841137
0 0.0 0.293752444893
3 0.0 0.152869073465
0 -1.0 -1.0
1 0.0 0.366756897032
2 0.0 0.269876223564
4 0.0 0.305627809972
5 0.0 0.219426479906
2 0.0 0.249861021727
4 0.0 0.581061282814
3 0.0 0.166868946031
1 0.0 0.435566877455
3 0.0 0.188422374889
3 0.0 0.371631670833
5 0.0 0.167555172846
5 0.0 0.485529661775
1 0.0 0.532686475396
2 0.0 0.465968312234
0 0.0 0.242536207706
1 0.0 0.410245974362
4 0.0 0.280129045844
1 0.0 0.0
3 0.0 0.371085791677
1 0.0 0.220171045974
2 0.0 0.568950714827
0 0.0 0.368887597203
2 0.0 0.39743134582
4 0.0 0.371569863081
2 0.0 0.091698840782
4 0.0 0.245000568509
4 1.0 10.0


[2017-08-23 12:00:05,045] Starting new video recorder writing to /home/z0m6ie/Documents/GitHub/Machine_Learning_Projects/deep-Q-learning/Test/Pong-v0/openaigym.video.0.24317.video000001.mp4


13.6776580811
1 episode, score = -20.0 
3 0.0 0.093969107084
0 0.0 0.269870477468
3 0.0 0.168395368248
2 0.0 0.15759556523
1 0.0 0.232242016509
3 0.0 0.217235773221
3 0.0 0.469689877242
0 0.0 0.433521832824
5 0.0 0.211834651038
1 0.0 0.279326706201
4 0.0 0.312638136864
5 0.0 0.366584305733
1 0.0 0.148336758077
4 0.0 0.158944632545
5 0.0 0.086403202638
5 0.0 0.347436466724
5 0.0 0.29784991774
5 0.0 0.103310064062
5 0.0 0.154372526184
1 0.0 0.467639771283
2 0.0 0.113506534405
2 0.0 0.186323620632
3 0.0 0.423777525187
2 0.0 0.200484101057
2 0.0 0.271931807667
0 0.0 0.0869845185354
0 0.0 0.34903382197
5 0.0 0.250832379967
0 0.0 0.421838143229
1 0.0 0.156262649506
4 0.0 0.565965186059
3 0.0 0.127185764372
0 1.0 10.0
15.8677351475
2 episode, score = -20.0 
4 0.0 0.236021384865
0 0.0 0.216484404132
5 0.0 0.454668152243
5 0.0 0.178589613095
5 0.0 0.315036462218
3 0.0 0.111273759149
4 0.0 0.259092378616
4 0.0 0.0847479919419
5 0.0 0.142401665747
5 0.0 0.170104370445
0 0.0 0.189734122351
5 0.0 9

[2017-08-23 12:00:22,901] Starting new video recorder writing to /home/z0m6ie/Documents/GitHub/Machine_Learning_Projects/deep-Q-learning/Test/Pong-v0/openaigym.video.0.24317.video000008.mp4


...

587.329326272
1012 episode, score = -21.0 
0 0.0 0.109271192461
0 0.0 0.145928265408
0 0.0 0.00767273429548
1 0.0 0.118878010601
1 0.0 0.321985844761
0 0.0 0.299161516339
0 0.0 0.0666921866387
0 0.0 0.00908185604028
0 0.0 0.169742708758
0 0.0 0.15284998928
0 0.0 0.0485122783072
0 0.0 0.329014541298
1 0.0 0.0618414690346
1 0.0 0.17534821929
0 0.0 0.146031858891
1 0.0 0.0817337103635
0 0.0 0.28750825426
0 0.0 0.0345468488261
0 0.0 0.288425664693
0 0.0 0.0606399208345
0 0.0 0.169968324915
0 0.0 0.0754924672097
0 0.0 0.153648518041
0 0.0 0.149259170294
0 0.0 0.262148438483
1 0.0 0.0443309920952
1 0.0 0.0739463653564
1 0.0 0.128945230991
0 0.0 0.246897733033
1 0.0 0.250945277363
1 0.0 0.118448705748
0 0.0 0.0756591114476
0 1.0 10.0
587.801535428
1013 episode, score = -21.0 
0 -1.0 -1.0
1 0.0 0.0209213787261
0 0.0 0.337388330519
0 0.0 0.0409289681017
1 0.0 0.00100292908971
0 0.0 0.244030177876
0 0.0 0.126382397577
0 0.0 0.0933197907507
5 0.0 0.268996505141
0 0.0 0.212199959859
0 0.0 0.096067

593.006335586
1024 episode, score = -21.0 
1 0.0 0.110567867585
3 0.0 0.312385278851
0 0.0 0.154395153299
0 0.0 0.102210728645
1 0.0 0.115306171596
1 0.0 0.032652203802
0 0.0 0.1844577768
1 0.0 0.09941793973
0 0.0 0.145054441944
0 0.0 0.206817341298
0 0.0 0.197309680536
1 0.0 0.053401901152
0 0.0 0.00195590012334
0 0.0 0.140928238615
0 0.0 0.180620426461
1 0.0 0.128611272633
0 0.0 0.197797235325
0 0.0 0.0393960182592
0 0.0 0.375466073692
1 0.0 0.362965158194
0 0.0 0.170349889532
0 0.0 0.173857568756
0 0.0 0.043104222849
0 0.0 0.170000389919
1 0.0 0.29270117721
0 0.0 0.0441720514968
1 0.0 0.0318934287988
1 0.0 0.0407592089139
0 0.0 -0.00359805929102
1 0.0 0.11944484961
0 0.0 0.174491485253
0 0.0 0.0464642043225
1 1.0 10.0
593.464774966
1025 episode, score = -21.0 
0 0.0 0.119382275216
1 0.0 0.125436599046
1 0.0 0.00203136229701
1 0.0 0.164255410016
1 0.0 0.0446931487508
1 0.0 0.140251032874
1 0.0 0.0361640697867
1 0.0 0.257190152764
1 0.0 0.281154292345
0 0.0 0.258295249164
0 0.0 0.2047

598.787847877
1036 episode, score = -21.0 
1 0.0 0.00119628288341
0 0.0 0.132763556689
1 0.0 0.151704610661
1 0.0 0.00119628288341
0 0.0 0.208129386485
0 0.0 0.192566531047
1 0.0 0.153294969365
1 0.0 0.0442382395305
0 0.0 0.104253957152
0 0.0 0.107404135399
0 0.0 0.165300946489
1 0.0 0.00119628288341
1 0.0 0.189884890392
0 0.0 0.147390617162
1 0.0 0.192893731043
1 0.0 0.108600633435
0 0.0 0.303943608284
1 0.0 0.235652979702
1 0.0 0.00119628288341
1 0.0 0.00119628288341
0 0.0 0.117343192533
0 0.0 0.0322519792579
1 0.0 0.0953400572017
1 0.0 0.265739450991
1 0.0 0.0158683940638
1 0.0 0.239448871806
1 0.0 0.173788779348
1 0.0 0.262599402845
1 0.0 0.251280813664
1 0.0 0.116343111254
1 0.0 0.19926845929
1 0.0 0.209022815153
1 1.0 10.0
599.261117101
1037 episode, score = -21.0 
1 0.0 0.188267929941
0 0.0 0.220351020858
1 0.0 0.107392687865
1 -1.0 -1.0
0 0.0 0.12611152719
1 0.0 0.261491686463
1 0.0 0.0725442955121
3 0.0 0.21854749091
1 0.0 0.18364579086
1 0.0 0.370554739237
1 0.0 0.36634621489

604.568350226
1048 episode, score = -21.0 
1 0.0 0.157609364793
1 0.0 0.00112085408752
1 0.0 0.102694070622
1 0.0 0.131756620288
0 0.0 0.315696102172
1 0.0 0.345841999412
1 0.0 0.250674972653
1 0.0 0.00112085408752
1 0.0 0.160740421638
1 0.0 0.0767559459731
1 0.0 0.193691962078
1 0.0 0.166864762992
0 0.0 0.00112085408752
1 0.0 0.00112085408752
0 0.0 0.239363930807
1 0.0 0.0257560712714
1 0.0 0.232222173125
1 0.0 0.253818861455
1 0.0 0.176927964091
1 0.0 0.118483703345
0 0.0 0.047582478784
1 0.0 0.214177763253
1 0.0 0.265694286078
1 0.0 0.0504362269342
1 0.0 0.0911192591339
1 0.0 0.13606220305
1 0.0 0.00112085408752
0 0.0 0.289002060682
1 0.0 0.367200388491
1 0.0 0.1368896707
1 0.0 0.0559910237007
1 0.0 0.18247506094
1 1.0 10.0
605.045201153
1049 episode, score = -21.0 
1 0.0 0.307000710189
1 0.0 0.291943168849
1 0.0 0.124013249293
1 0.0 0.00116940550832
1 0.0 0.00116940550832
1 0.0 0.11347548911
1 0.0 0.129737552181
1 0.0 0.154177054703
1 0.0 0.216488155469
0 0.0 0.3136125319
1 0.0 0.1

610.370319158
1060 episode, score = -21.0 


In [ ]:
import gym
from gym import wrappers
gym.upload(env_name, api_key='sk_WRCITkqmTJKYB9hvBk5tPA')

In [ ]:
processed_observation.shape

In [ ]:
reward

In [ ]:
test = agent.model.predict(state)

In [ ]:
test[0]

In [ ]:
env.reset()
env.close()

In [ ]:
len(agent.memory)

In [ ]:
processed_observation = Image.fromarray(state, 'RGB')
processed_observation = processed_observation.convert('L')
processed_observation = processed_observation.resize((80, 80))
processed_observation = np.array(processed_observation)
processed_observation = processed_observation.reshape(processed_observation.shape[0], processed_observation.shape[1], 1)

In [ ]:
new_state

In [ ]:
def RGBprocess(new_state):
        processed_observation = Image.fromarray(new_state, 'RGB')
        processed_observation = processed_observation.convert('L')
        processed_observation = processed_observation.resize((80, 80))
        processed_observation = np.array(processed_observation)
        processed_observation = processed_observation.reshape(processed_observation.shape[0], processed_observation.shape[1], 1) #1x80x80x1
        return processed_observation

In [ ]:
new_state_dif = new_state_dif.reshape(new_state_dif.shape[1], new_state_dif.shape[2])
img = Image.fromarray(new_state_dif, 'L')

In [ ]:
#img = Image.fromarray(state, 'L')
img.show()

In [ ]:
thing = [1,2,3,4,5,6]
for x in thing[-2:]:
    print(x)

In [ ]:
        # way to adjust reward, think incorrect
        if reward == 0:
            num_actions += 1
            short_mem.append([state, action, reward, new_state_dif, done])
        else:
            num_actions += 1
            short_mem.append([state, action, reward, new_state_dif, done])
            if reward == -1.0:
                for m in short_mem:
                    m[2] = -1.0
                    agent.remember(m[0], m[1], m[2], m[3], m[4])
                num_actions = 0
            elif reward == 1.0:
                for m in short_mem:
                    m[2] = 1.0
                    agent.remember(m[0], m[1], m[2], m[3], m[4])
            short_mem = []

In [ ]:
        # early attempt at increasing samples with positive reward
        winsample = [s for s in self.memory if s[2]== 1.0]
        #print(winsample)
        tuple(winsample)
        if len(winsample) > 4:
            Samplewin = random.sample(winsample, 4)
            Sample += Samplewin
        else:  
            Sample += winsample